<a href="https://colab.research.google.com/github/poketmonster/sentimentanalisys/blob/master/Entropia_ALC_SentimentAnalisys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv, operator
import numpy as np
import nltk
import string
import re
import math
import random
import xml.etree.ElementTree as et
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from random import shuffle
from nltk.stem.porter import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:
from google.colab import files

uploaded = files.upload() # then browse, select the files. It's then uploaded



SyntaxError: ignored

In [0]:
!wget https://github.com/poketmonster/sentimentanalisys/blob/master/TASS2017_T1_development.xml

--2019-04-12 15:13:46--  https://github.com/poketmonster/sentimentanalisys/blob/master/TASS2017_T1_development.xml
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-12 15:13:46 ERROR 404: Not Found.



In [0]:
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas DataFrame 
    with the given columns. The first element of df_cols is supposed to be 
    the identifier variable, which is an attribute of each node element in 
    the XML data; other features will be parsed from the text content of 
    each sub-element. """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    out_df = pd.DataFrame(columns = df_cols)
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.findall(el) is not None:
                res.append(node.findall(el).text)
            else: 
                res.append(None)
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index = True)
        
    return out_df

In [0]:
train = parse_XML("TASS2017_T1_training.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print(train.head())

dev = parse_XML("TASS2017_T1_development.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print(dev.head())

test = parse_XML("TASS2017_T1_test.xml", ["tweetid", "user", "content", "date", "lang", "./sentiment/polarity/value"])
print(test.head())

,tweetid,user,content,date,lang,./sentiment/polarity/value
0,None,wikimiscojones,@LonelySoad mientras que no te pillen la prime...,2016-08-30 10:24:55,es,None
1,None,HLF_Metr4spt,@ceemeese ya era hora de volver al csgo y deja...,2016-08-30 06:08:17,es,None
2,None,AVazquez_C,@mireiaescribano justo cuando se terminan las ...,2016-08-30 06:06:12,es,None
3,None,minniecris,@LuisMartinez22_ pensba q iba a hacer @wxplosi...,2016-08-30 12:32:05,es,None
4,None,VI_Lelouch,"@Vic_Phantomhive Si lo encuentro, sin compañer...",2016-08-30 12:32:02,es,None


In [0]:


tweetsen=open('en_AMI_TrainingSet.csv', 'r')
tweetsen = csv.reader(tweetsen, delimiter=";")
tweetsen=list(tweetsen)

tweetses=open('es_AMI_TrainingSet.csv', 'r')
tweetses = csv.reader(tweetses, delimiter=";")
tweetses=list(tweetses)

language="EN"
random.seed(9001)


[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /content/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [0]:

total=tweetsen
tknzr = TweetTokenizer()

positiveWords=()
negativeWords=()


otherstopwords=["http", "https", ",", ".", ";", ":", "@", "RT"]

size=len(total)
print(size)
print(total[2][2])

3563
1


In [0]:
for i in range(0,size):
    #print(total[i][1])
    senseurl = re.sub(r"http\S+", "", total[i][1])
    tokenitzat = tknzr.tokenize(senseurl)
    #print(tokenitzat)
    if(total[i][2]=="0"): #No misoginos
        #print("NO missogin "+total[i][0])
        for word in tokenitzat:
            if word not in string.punctuation:
                if word not in stopwords.words('english') and word not in otherstopwords:
                    positiveWords+=(word.lower(), )
            
    elif(total[i][2]=="1"): #Misoginos
        #print("missogin "+total[i][0])
        for word in tokenitzat:
            if word not in string.punctuation:
                if word not in stopwords.words('english') and word not in otherstopwords:
                    negativeWords+=(word.lower(), )

print("positive")
print(positiveWords)
print("negative")
print(negativeWords)

positiveWordfreq = {}
negativeWordfreq= {}

for word in positiveWords:
    count = positiveWordfreq.get(word,0)
    positiveWordfreq[word] = count + 1

for word in negativeWords:
    count = negativeWordfreq.get(word,0)
    negativeWordfreq[word] = count + 1


#print (str(negativeWordfreq))
#print (str(positiveWordfreq))

print(len(positiveWords))
print(len(negativeWords))

print(len(positiveWordfreq))
print(len(negativeWordfreq))

#Preparem training set

positive
('@bdotchristian', '@bheescott', 'bitch', 'stop', 'tryna', 'hard', 'hoe', 'tell', 'love', 'tranny', 'tight', 'pussy', 'porn', 'quick', 'fucking', 'dick', 'sucking', 'my', 'sister', '#nobraday', 'raise', 'cancer', 'awareness', 'hoe', '😅', '@kingcarlos_17', 'hahahahahaha', 'hell', 'nah', 'nigga', 'even', 'make', 'quote', 'said', 'fat', 'bitch', 'i', 'love', 'religious', 'brothers', 'sisters', '@realdonaldtrump', 'fuck', 'you', "you're", 'not', 'even', 'a', 'real', 'theocrat', 'you', 'fat', 'usless', 'bitch', 'i', 'sick', 'every', 'country', 'universal', 'hc', 'argument', 'bitch', '...', 'reason', 'afford', '…', '@hansimpkin', 'wait', 'till', 'got', 'gin', 'tomorrow', 'cock', 'i', 'think', 'girl', 'tried', 'blackmail', 'nelly', 'failed', 'cooked', 'rape', 'story', 'moment', 'broke', 'hoe', 'panic', 'consistently', 'bitch', 'about', 'liberals', 'consider', 'yourself', 'unfollowed', 'if', 'you', 'consistently', 'post', 'bullshit', 'politics', 'fuck', 'you', 'this', 'is', 'a', 'plac

In [0]:
def create_word_features(tweet, lang):
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    for word in tokenitzat:
            if word not in string.punctuation and word not in otherstopwords:
                if lang=="EN" and word not in stopwords.words('english'):
                    usables.append(word.lower())
                if lang=="ES" and word not in stopwords.words('spanish'):
                    usables.append(word.lower())
    
    diccionario=dict([word, True] for word in usables)
    #diccionario.update([lang, True])
    return diccionario
  
def create_word_featuresSTM(tweet, lang):
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    for word in tokenitzat:
            if word not in string.punctuation and word not in otherstopwords:
                if lang=="EN" and word not in stopwords.words('english'):
                    usables.append(word.lower())
                if lang=="ES" and word not in stopwords.words('spanish'):
                    usables.append(word.lower())
    
    stemmer = PorterStemmer()
    diccionario=dict([stemmer.stem(word), True] for word in usables)
    #diccionario.update([lang, True])
    return diccionario

def bigrams(tweet, lang):
    bigrams=[]
    i=0
    prev=False
    for word in tknzr.tokenize(tweet):
        if  word not in otherstopwords:
            if(not prev):
                prev=word
            else:
                bigrams.append(prev+" "+word)
                prev=word
            i+=1

    diccionario=dict([word, True] for word in bigrams)
    return diccionario

def trigrams(tweet, lang):
    trigrams=[]
    i=0
    word1=False
    word2=False
    for wordact in tknzr.tokenize(tweet):
        if  wordact not in otherstopwords:
            if(not word2):
                word2=wordact
            elif(not word1):
                word1=word2
                word2=wordact
            else:
                trigrams.append(word1+" "+word2+" "+wordact)
                word1=word2
                word2=wordact
            i+=1
            #print("Actual: "+str(word1)+" "+str(word2)+" "+str(wordact))

    diccionario=dict([word, True] for word in trigrams)
    return diccionario

def bigramaCaracter(tweet):
    charbigrams=[]
    bigrams = nltk.bigrams(tweet)
    for bi in bigrams:
        charbigrams.append(bi[0]+" "+bi[1])
    #print (str(charbigrams))

    diccionario=dict([word, True] for word in charbigrams)
    return diccionario
  
def posWordStm(tweet, lang):
    stemmer = PorterStemmer()
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    stemizat = [stemmer.stem(word) for word in tokenitzat]
    
    for word in stemizat:
      if word not in string.punctuation and word not in otherstopwords:
        if lang=="EN" and word not in stopwords.words('english'):
          usables.append(word.lower())
        if lang=="ES" and word not in stopwords.words('spanish'):
          usables.append(word.lower())
          
          
    pos= nltk.pos_tag(usables)

    diccionario=dict( word for word in pos)
    return diccionario
  
def posWord(tweet, lang):
    stemmer = PorterStemmer()
    usables = []
    tokenitzat = tknzr.tokenize(tweet)
    
    for word in tokenitzat:
      if word not in string.punctuation and word not in otherstopwords:
        if lang=="EN" and word not in stopwords.words('english'):
          usables.append(word.lower())
        if lang=="ES" and word not in stopwords.words('spanish'):
          usables.append(word.lower())
    
    
    pos= nltk.pos_tag(usables)

    diccionario=dict( word for word in pos)
    return diccionario
  

def  features(tweet, lang):
    #Traiem fora les URL
    tweet=re.sub(r"http\S+", "", tweet)
    #Eliminem repeticions de la mateixa lletra més de 2 cops
    tweet=re.sub(r'(.)\1+', r'\1\1', tweet) 
    #Eliminem referències a usuaris
    tweet=re.sub(r"@\S+", "", tweet)
    
    words=create_word_featuresSTM(tweet, lang)
    #words=create_word_features(tweet, lang)
    #words=posWordStm(tweet, lang)
    #words=posWord(tweet, lang)
    #Twbigrams=bigrams(tweet, lang)
    #Twtrigrams=trigrams(tweet, lang)
    #charbigrams=bigramaCaracter(tweet)

    diccionario = words.copy()
    #diccionario = Twbigrams.copy()
    #diccionario.update(Twbigrams)
    #diccionario.update(Twtrigrams)
    #diccionario.update(charbigrams)

    return diccionario

In [0]:
for i in range(3196,3198):
    print (total[i][1])
    print(features(total[i][1], language))


Here's an actual picture of me in kindergarten tryin to get girls to play dr https://t.co/yl64i5cmBV
{"here'": True, 'actual': True, 'pictur': True, 'kindergarten': True, 'tryin': True, 'get': True, 'girl': True, 'play': True, 'dr': True}
RT @Scouse_ma: @Yourfuckboy1 you are VILEEEE 😭
{'vile': True, '😭': True}


In [0]:
for i in range(3196,3198):
    print (total[i][1])
    print(features(total[i][1] , language),0)
    

misogin_tweets = []
nomisogin_tweets = []

print("total")
print(size)
for i in range(0,size):
    if(total[i][2]=="1"):
        misogin_tweets.append((features(total[i][1] , language),1))
    if(total[i][2]=="0"):
        nomisogin_tweets.append((features(total[i][1] , language),0))
       

total_set=misogin_tweets+nomisogin_tweets
print ("long total")
print (len(total_set))

shuffle(total_set)

trainlen=int(len(total_set)*0.8)

train_set=total_set[0:trainlen]

iniTest=trainlen+1
fin=len(total_set)-1
test_set=total_set[iniTest:fin]

print ("long train")
print (len(train_set))

print ("long test")
print (len(test_set))


maxent_classifier = nltk.classify.MaxentClassifier.train(train_set,max_iter=25)

Here's an actual picture of me in kindergarten tryin to get girls to play dr https://t.co/yl64i5cmBV
{"here'": True, 'actual': True, 'pictur': True, 'kindergarten': True, 'tryin': True, 'get': True, 'girl': True, 'play': True, 'dr': True} 0
RT @Scouse_ma: @Yourfuckboy1 you are VILEEEE 😭
{'vile': True, '😭': True} 0
total
3563
long total
3562
long train
2849
long test
711
  ==> Training (25 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.503
             2          -0.55238        0.841
             3          -0.47196        0.878
             4          -0.41838        0.896
             5          -0.37933        0.909
             6          -0.34915        0.920
             7          -0.32486        0.927
             8          -0.30473        0.931
             9          -0.28767        0.937
            10          -0.27297        0.942
            11          -0.26011        0.9

In [0]:
for tweet in test_set:
    #print(tweet[1])
    test_class = maxent_classifier.classify(tweet[0])
    class_prob = maxent_classifier.prob_classify(tweet[0])
    #print("calculada: "+str(test_class)+"  --  real: "+str(tweet[1]))
    #if str(test_class)!=str(tweet[1]):
        #for label in class_prob.samples():
            #print("%s: %f" % (label, class_prob.prob(label)))
   

accuracy = nltk.classify.util.accuracy(maxent_classifier, test_set)

print("Accuracy: ")
print(accuracy*100)


print("Features: ")
print(maxent_classifier.show_most_informative_features(100))


Accuracy: 
70.88607594936708
Features: 
   4.644 #shitpussiessay==True and label is 0
   4.644 neverr==True and label is 0
   4.644 #yesallwomenbelonginthekitchen==True and label is 1
   4.644 #feminismisaw==True and label is 1
  -4.522 ram==True and label is 1
  -4.352 slag==True and label is 1
   3.872 fuckhead==True and label is 0
   3.854 megan==True and label is 1
   3.728 mercuri==True and label is 0
   3.534 hyperbol==True and label is 0
   3.381 stingray==True and label is 0
   3.308 #thingstimhowardcouldsav==True and label is 0
   3.286 #allin==True and label is 0
   3.224 boner==True and label is 1
   3.170 #cyberbulli==True and label is 1
   3.150 h20==True and label is 1
   3.140 homophob==True and label is 0
   3.071 bent==True and label is 0
   3.055 unlucki==True and label is 0
   3.041 carb==True and label is 0
   2.971 std==True and label is 0
   2.956 flag==True and label is 0
   2.945 balloon==True and label is 0
   2.931 julia==True and label is 1
   2.880 tay==True